**Análisis de abandono de usuarios de Waze: comprendiendo la retención de conductores**

Este proyecto investiga la deserción de usuarios (churn) para Waze, una popular aplicación de navegación. El objetivo es analizar un conjunto de datos de actividad del usuario para comprender patrones y características que distinguen a los usuarios que abandonan la aplicación (churned) de aquellos que permanecen activos (retained). Este análisis proporcionará información para ayudar a Waze a mejorar las estrategias de retención de usuarios.

**Exploración y preprocesamiento de datos**

El conjunto de datos proporcionado contiene información sobre 14,999 usuarios de Waze, incluyendo su comportamiento de conducción, patrones de uso de la aplicación y tipo de dispositivo. Las variables clave en el conjunto de datos incluyen:

* sessions: número de veces que un usuario abrió la aplicación.

* drives: número de veces que un usuario condujo al menos 1 km.

* total_sessions: una estimación del modelo del número total de sesiones desde que un usuario se registró.

* n_days_after_onboarding: número de días desde que un usuario se registró en la aplicación.

* total_navigations_fav1: total de navegaciones desde el registro al lugar favorito del usuario.

* total_navigations_fav2: total de navegaciones desde el registro al segundo lugar favorito del usuario.

* driven_km_drives: total de kilómetros conducidos usando la aplicación.

* duration_minutes_drives: duración total conducida en minutos usando la aplicación.

* activity_days: número de días que el usuario abre la aplicación.

* driving_days: número de días que el usuario conduce (al menos 1 km).

* device: tipo de dispositivo utilizado por el usuario (Android o iPhone).

* label: variable objetivo binaria; indica si el usuario abandonó (churned) o fue retenido (retained).

Primero, cargamos el conjunto de datos y examinamos su estructura:

In [17]:
import pandas as pd
import numpy as np

# Cargar el dataset
df = pd.read_csv('waze_dataset.csv')

In [2]:
# Inspeccionar los primeros registros
df.head()

,ID,label,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days,device
0,0,retained,283,226,296.748273,2276,208,0,2628.845068,1985.775061,28,19,Android
1,1,retained,133,107,326.896596,1225,19,64,13715.920550,3160.472914,13,11,iPhone
2,2,retained,114,95,135.522926,2651,0,0,3059.148818,1610.735904,14,8,Android
3,3,retained,49,40,67.589221,15,322,7,913.591123,587.196542,7,3,iPhone
4,4,retained,84,68,168.247020,1562,166,5,3950.202008,1219.555924,27,18,Android


In [3]:
# Obtener información general del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       14999 non-null  int64  
 1   label                    14299 non-null  object 
 2   sessions                 14999 non-null  int64  
 3   drives                   14999 non-null  int64  
 4   total_sessions           14999 non-null  float64
 5   n_days_after_onboarding  14999 non-null  int64  
 6   total_navigations_fav1   14999 non-null  int64  
 7   total_navigations_fav2   14999 non-null  int64  
 8   driven_km_drives         14999 non-null  float64
 9   duration_minutes_drives  14999 non-null  float64
 10  activity_days            14999 non-null  int64  
 11  driving_days             14999 non-null  int64  
 12  device                   14999 non-null  object 
dtypes: float64(3), int64(8), object(2)
memory usage: 1.5+ MB


Este código nos da una visión general de las variables, sus tipos de datos y la presencia de valores faltantes. Observamos que la variable label tiene 700 valores faltantes (aproximadamente 4.7% de los datos).

**Análisis de los datos faltantes**

Para entender si los valores faltantes en label eran aleatorios o sistemáticos, comparamos los usuarios con etiquetas faltantes con aquellos con datos completos.

In [4]:
# Aislar filas con valores nulos en 'label'
null_df = df[df['label'].isnull()]

# Estadísticas descriptivas de filas con 'label' nulo
null_df.describe()

,ID,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days
count,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000
mean,7405.584286,80.837143,67.798571,198.483348,1709.295714,118.717143,30.371429,3935.967029,1795.123358,15.382857,12.125714
std,4306.900234,79.987440,65.271926,140.561715,1005.306562,156.308140,46.306984,2443.107121,1419.242246,8.772714,7.626373
min,77.000000,0.000000,0.000000,5.582648,16.000000,0.000000,0.000000,290.119811,66.588493,0.000000,0.000000
25%,3744.500000,23.000000,20.000000,94.056340,869.000000,4.000000,0.000000,2119.344818,779.009271,8.000000,6.000000
50%,7443.000000,56.000000,47.500000,177.255925,1650.500000,62.500000,10.000000,3421.156721,1414.966279,15.000000,12.000000
75%,11007.000000,112.250000,94.000000,266.058022,2508.750000,169.250000,43.000000,5166.097373,2443.955404,23.000000,18.000000
max,14993.000000,556.000000,445.000000,1076.879741,3498.000000,1096.000000,352.000000,15135.391280,9746.253023,31.000000,30.000000


In [5]:

# Estadísticas descriptivas de filas sin valores nulos
not_null_df = df[~df['label'].isnull()]
not_null_df.describe()

,ID,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days
count,14299.000000,14299.000000,14299.000000,14299.000000,14299.000000,14299.000000,14299.000000,14299.000000,14299.000000,14299.000000,14299.000000
mean,7503.573117,80.623820,67.255822,189.547409,1751.822505,121.747395,29.638296,4044.401535,1864.199794,15.544653,12.182530
std,4331.207621,80.736502,65.947295,136.189764,1008.663834,147.713428,45.350890,2504.977970,1448.005047,9.016088,7.833835
min,0.000000,0.000000,0.000000,0.220211,4.000000,0.000000,0.000000,60.441250,18.282082,0.000000,0.000000
25%,3749.500000,23.000000,20.000000,90.457733,878.500000,10.000000,0.000000,2217.319909,840.181344,8.000000,5.000000
50%,7504.000000,56.000000,48.000000,158.718571,1749.000000,71.000000,9.000000,3496.545617,1479.394387,16.000000,12.000000
75%,11257.500000,111.000000,93.000000,253.540450,2627.500000,178.000000,43.000000,5299.972162,2466.928876,23.000000,19.000000
max,14998.000000,743.000000,596.000000,1216.154633,3500.000000,1236.000000,415.000000,21183.401890,15851.727160,31.000000,30.000000


Las estadísticas descriptivas (media, desviación estándar) para las variables numéricas fueron muy similares entre los dos grupos, lo que sugiere que no hay diferencias significativas en sus patrones de conducción o uso de la aplicación.

Profundizamos en el análisis de datos faltantes, desglosando por tipo de device:

In [6]:
# Conteo de valores nulos por dispositivo
null_df['device'].value_counts()

device
iPhone     447
Android    253
Name: count, dtype: int64

In [7]:
# Porcentaje de valores nulos por dispositivo
null_df['device'].value_counts(normalize=True)

device
iPhone     0.638571
Android    0.361429
Name: proportion, dtype: float64

In [8]:
# Porcentaje de usuarios de iPhone y Android en el conjunto de datos completo
df['device'].value_counts(normalize=True)

device
iPhone     0.644843
Android    0.355157
Name: proportion, dtype: float64

La distribución de usuarios de iPhone (63.9%) y Android (36.1%) con etiquetas faltantes fue similar a la del conjunto de datos completo (iPhone: 64.5%, Android: 35.5%). Esta consistencia indica que los datos probablemente faltan al azar y no introducen sesgo basado en el tipo de dispositivo.

Análisis de churn: identificando diferencias clave

El conjunto de datos indica que el 17.7% de los usuarios abandonaron (churned), mientras que el 82.3% fueron retenidos.

In [9]:
# Calcular el conteo de usuarios retenidos y churned
df['label'].value_counts()

label
retained    11763
churned      2536
Name: count, dtype: int64

In [10]:
# Calcular el porcentaje de usuarios retenidos y churned
df['label'].value_counts(normalize=True)

label
retained    0.822645
churned     0.177355
Name: proportion, dtype: float64

Para descubrir posibles impulsores de la deserción, realizamos un análisis comparativo entre estos dos grupos, centrándonos en los valores medianos para mitigar la influencia de valores atípicos.

In [11]:
# Calcular la mediana de todas las columnas para usuarios retenidos y churned
df.groupby('label').median(numeric_only=True)

,ID,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days
label,,,,,,,,,,,
churned,7477.5,59.0,50.0,164.339042,1321.0,84.5,11.0,3652.655666,1607.183785,8.0,6.0
retained,7509.0,56.0,47.0,157.586756,1843.0,68.0,9.0,3464.684614,1458.046141,17.0,14.0


Hallazgos clave:

1. Comportamiento de conducción: los usuarios que abandonaron exhibieron patrones de conducción distintos:

* Mayor kilometraje diario: el usuario mediano que abandonó condujo significativamente más kilómetros por día de conducción (698 km) en comparación con el usuario mediano retenido (290 km).

In [12]:
# Añadir una columna a df llamada km_per_driving_day
df['km_per_driving_day'] = df['driven_km_drives'] / df['driving_days']

# Agrupar por label, calcular la mediana y aislar para km por día de conducción
median_km_per_driving_day = df.groupby('label').median(numeric_only=True)[['km_per_driving_day']]
median_km_per_driving_day

,km_per_driving_day
label,
churned,697.541999
retained,289.549333


* Mayor distancia por viaje: el usuario promedio que abandonó la app realizaba viajes más largos (74.11 km) que el usuario promedio que fue retenido (75.01 km)

In [13]:
# Añadir una columna a df llamada km_per_drive
df['km_per_drive'] = df['driven_km_drives'] / df['drives']

# Agrupar por label, calcular la mediana y aislar para km por viaje
median_km_per_drive = df.groupby('label').median(numeric_only=True)[['km_per_drive']]
median_km_per_drive

,km_per_drive
label,
churned,74.109416
retained,75.014702


* Menos días de conducción, más viajes: los usuarios que abandonaron tuvieron menos días de conducción pero más viajes por día, lo que sugiere viajes más largos y menos frecuentes.

In [14]:
# Añadir una columna a df llamada drives_per_driving_day
df['drives_per_driving_day'] = df['drives'] / df['driving_days']

# Agrupar por label, calcular la mediana y aislar para viajes por día de conducción
median_drives_per_driving_day = df.groupby('label').median(numeric_only=True)[['drives_per_driving_day']]
median_drives_per_driving_day

,drives_per_driving_day
label,
churned,10.0000
retained,4.0625


* Duraciones de viaje más largas: los usuarios que abandonaron pasaron más tiempo conduciendo en general.

2. Uso de la aplicación:

* Menor compromiso: los usuarios retenidos usaron la aplicación en más del doble de días que los usuarios que abandonaron.

* Número similar de viajes: los usuarios que abandonaron tuvieron un número ligeramente mayor de viajes en el último mes que los usuarios retenidos.

3. Tipo de dispositivo:

* Sin diferencia significativa: la tasa de abandono fue casi idéntica tanto para usuarios de iPhone (17.6%) como de Android (17.8%), lo que sugiere que el tipo de dispositivo no es un factor significativo en el abandono. Esto se puede observar al ver que hay una cantidad similar de usuarios de Android y iPhone para cada etiqueta (churned o retained):

In [15]:
# Para cada etiqueta (churned/retained), calcular el número de usuarios de Android y iPhone
df.groupby(['label', 'device']).size()

label     device 
churned   Android     891
          iPhone     1645
retained  Android    4183
          iPhone     7580
dtype: int64

In [16]:
# Para cada etiqueta, calcular el porcentaje de usuarios de Android y iPhone
df.groupby('label')['device'].value_counts(normalize=True)

label     device 
churned   iPhone     0.648659
          Android    0.351341
retained  iPhone     0.644393
          Android    0.355607
Name: proportion, dtype: float64

**Perfil potencial del usuario:**

El análisis sugiere que los usuarios que abandonaron pueden representar un perfil de usuario distinto, que potencialmente incluye conductores de larga distancia o individuos con necesidades de viaje menos frecuentes pero de mayor distancia. Sus patrones de conducción (alto kilometraje diario y duraciones de viaje más largas), junto con una menor frecuencia de uso de la aplicación, sugieren que Waze podría no satisfacer plenamente sus necesidades específicas en comparación con los usuarios más típicos que viajan distancias más cortas con mayor regularidad.

**Recomendaciones**

Investigación adicional de superconductores: el alto volumen de conducción de los usuarios que abandonaron justifica una mayor investigación. Recopilar más datos sobre estos "superconductores" podría revelar puntos débiles específicos o necesidades insatisfechas que contribuyen al abandono. Comprender sus razones para usar Waze y sus contextos de conducción podría llevar a mejoras específicas.

Adaptar funciones para diferentes segmentos de usuarios: Waze debería considerar desarrollar o mejorar funciones que satisfagan las necesidades de los usuarios con patrones de conducción de alto kilometraje y larga distancia. Esto podría incluir funciones relacionadas con la planificación de viajes de larga distancia, la optimización de paradas de descanso u opciones de ruta especializadas.

Investigar la recopilación de datos: es esencial comprender cómo se recopilaron los datos para determinar si la alta proporción de conductores pesados es representativa de la base general de usuarios de Waze o un resultado del sesgo de muestreo.

**Conclusión**

Este análisis proporciona información valiosa sobre la deserción de usuarios en Waze. Al resaltar las características distintivas de los usuarios que abandonaron, en particular su comportamiento de conducción, el estudio sienta las bases para intervenciones específicas para mejorar la retención de usuarios. Abordar las necesidades específicas de los conductores de alto kilometraje y comprender el contexto de su uso de la aplicación podría tener un impacto significativo en la capacidad de Waze para retener este importante segmento de usuarios. Investigaciones adicionales, incluidos datos cualitativos de encuestas o entrevistas a usuarios, mejorarían aún más nuestra comprensión y contribuirían a una estrategia de retención de usuarios más completa.